In [2]:
using RCall # use R in Julia and thus in this jupyter notebook
using CSV # save/load txt files
using LowRankApprox # randomized low-rank approximation

In [6]:
L1 = Array{Float64,2}(CSV.read("../data/sample5000x20.txt", nullable = false, header = false, delim = ' '));

In [5]:
include("../code/julia/mixSQP_time.jl")
include("../code/julia/mixEM.jl");
include("../code/julia/REBayes.jl");
include("../code/julia/makedata.jl");

In [20]:
srand(1)
temp = pqrfact(L1, rank = 5);
L = temp[:Q] * temp[:R] * temp[:P]';

In [21]:
[rank(L1) rank(L)]

1×2 Array{Int64,2}:
 17  5

In [22]:
# time comparison between mixEM, mixSQP and REBayes

## mixEM;
tic(); x_em = mixEM(L)[1]; t_em = toq();

x_rebayes,t_rebayes = REBayes(L);

tic(); x_mixsqp = mixSQP_time(L)[1]; t_mixsqp = toq();

# mixSQP ourperforms on this small dataset 5000x20
["mixEM" "REbayes" "mixSQP"; t_em t_rebayes t_mixsqp]

2×3 Array{Any,2}:
   "mixEM"   "REbayes"   "mixSQP"
 17.3617    0.081       0.0420426

In [23]:
# EM does not converge until maxiter
[x_em x_rebayes x_mixsqp]

20×3 Array{Float64,2}:
 0.358333      0.35897    0.447119 
 0.00397846    0.0        0.0      
 0.000109288   0.0        0.0      
 1.99013e-7    0.0        0.0      
 4.40543e-12   0.0        0.0      
 1.63095e-19   0.0        0.0      
 5.72183e-30   0.0        0.0      
 6.50616e-41   0.0        0.0      
 7.34051e-44   0.0        0.0      
 8.55295e-28   0.0        0.0      
 0.251122      0.267272   0.0369685
 0.0592966     0.0433985  0.111889 
 2.28634e-24   0.0        0.0755728
 0.281933      0.286772   0.32845  
 0.0452267     0.0435877  0.0      
 1.39568e-228  0.0        0.0      
 1.97626e-323  0.0        0.0      
 4.94066e-324  0.0        0.0      
 0.0           0.0        0.0      
 0.0           0.0        0.0      

In [25]:
minf = min.(eval_f(L1,x_em), eval_f(L1,x_rebayes),  eval_f(L1,x_mixsqp));
[eval_f(L1,x_em), eval_f(L1,x_rebayes),  eval_f(L1,x_mixsqp)]

3-element Array{Float64,1}:
 0.470621
 0.470621
 0.467781